## Messari standardised subgraphs

https://github.com/messari/subgraphs

### Solarbeam - https://docs.solarbeam.io/faq

In [2]:
import pandas as pd
import numpy as np
import time
import requests
import datetime as dt

In [3]:
endpoint = 'https://api.thegraph.com/subgraphs/name/messari/solarbeam-moonriver'

6-Month Average Daily Volume

In [4]:
query = """{
  financialsDailySnapshots(first: 183, orderBy: timestamp, orderDirection: desc){
    id
    protocol{
      name
    }
    dailyVolumeUSD
    dailyTotalRevenueUSD
    dailySupplySideRevenueUSD
    dailyProtocolSideRevenueUSD
    timestamp
    blockNumber
  }
}
"""

In [5]:
request = requests.post(endpoint, json={'query':query})
data = request.json()

In [6]:
df = pd.json_normalize(data['data']['financialsDailySnapshots'])

In [7]:
df['dailyVolumeUSD'].astype(float).mean()

3310237.488307767

6-Month Average TVL

In [9]:
#DefiLlama https://defillama.com/protocols/dexes 
lma = pd.read_csv("https://api.llama.fi/dataset/solarbeam.csv")

In [10]:
lma

,Unnamed: 0,Date,Timestamp,Solarbeam,Solarbeam.1,Solarbeam.2,Solarbeam.3,Solarbeam.4,Solarbeam.5,Solarbeam.6,...,Solarbeam.32,Solarbeam.33,Solarbeam.34,Solarbeam.35,Solarbeam.36,Solarbeam.37,Solarbeam.38,Solarbeam.39,Solarbeam.40,Solarbeam.41
0,NaN,NaN,NaN,Dexes,Dexes,Dexes,Dexes,Dexes,Dexes,Dexes,...,Dexes,Dexes,Dexes,Dexes,Dexes,Dexes,Dexes,Dexes,Dexes,Dexes
1,NaN,NaN,NaN,Total,moonriver,Total,Total,Total,Total,Total,...,moonriver,moonriver,moonriver,moonriver,moonriver,moonriver,moonriver,moonriver,moonriver,moonriver
2,NaN,NaN,NaN,TVL,TVL,Tokens(USD),Tokens(USD),Tokens(USD),Tokens(USD),Tokens(USD),...,Tokens,Tokens,Tokens,Tokens,Tokens,Tokens,Tokens,Tokens,Tokens,Tokens
3,NaN,NaN,NaN,NaN,NaN,USDT,moonriver,kusama,frax,tether,...,USDT,moonriver,kusama,frax,tether,magic-internet-money,binance-usd,usd-coin,bitcoin,mimatic
4,NaN,04/09/2021,1.630714e+09,324457.32223,324457.32223,324457.32223,NaN,NaN,NaN,NaN,...,324457.32223,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
301,NaN,14/07/2022,1.657757e+09,28461440.367345914,28461440.367345914,NaN,NaN,2643393.8337562983,4254053.773291846,1497841.581498,...,NaN,NaN,53858.880068384235,4254053.773291846,1497841.581498,13807.563647947374,1599473.4762276385,18107119.951185003,15.169546539999999,12925.43129256925
302,NaN,15/07/2022,1.657843e+09,28884345.0476636,28884345.0476636,NaN,NaN,2767076.3041666993,4286519.62034963,1504180.515069,...,NaN,NaN,53771.40116919353,4288543.81302938,1504180.515069,13298.210665077342,1581088.3172103465,18388780.116411,15.18552451,12872.519871707229
303,NaN,16/07/2022,1.657930e+09,29449453.66706093,29449453.66706093,NaN,NaN,3023641.383569109,4337876.53026205,1497503.3321958708,...,NaN,NaN,55479.658414112084,4341979.701079571,1496007.324871,12929.133892575117,1565125.9972548452,18680548.02257,15.23401261,12947.800295072697
304,NaN,17/07/2022,1.658016e+09,30363374.27842397,30363374.27842397,NaN,NaN,3149329.881285669,4331279.594472345,1490856.749114,...,NaN,NaN,55563.33594364271,4339055.181357337,1490856.749114,10952.886643002565,1553833.3544719461,19493407.330275,15.399206190000001,12895.986060362737


In [11]:
lma.iloc[:,3]

0                   Dexes
1                   Total
2                     TVL
3                     NaN
4            324457.32223
              ...        
301    28461440.367345914
302      28884345.0476636
303     29449453.66706093
304     30363374.27842397
305      29895708.6068861
Name: Solarbeam, Length: 306, dtype: object

In [12]:
#Just eth currently can take total by changing the header
lma.iloc[-183:, 3].astype(float).mean()

27671389.253872823

Largest Pool Concentration

In [13]:
df = pd.DataFrame()
ts = 0

while True:
    sl = len(df)
    query = """
{
  liquidityPools(first:1000, orderBy: createdTimestamp, orderDirection: asc, where: {createdTimestamp_gt: """+f'"{ts}"'+"""}){
    id
    name
    totalValueLockedUSD
    cumulativeVolumeUSD
    createdTimestamp
    createdBlockNumber
  }
}
"""
    request = requests.post(endpoint, json={'query':query})
    data = request.json()

    pool = pd.json_normalize(data['data']['liquidityPools'])
    df = df.append(pool)
    df.drop_duplicates(inplace=True)
    if len(df) == sl:
      break
    ts = df['createdTimestamp'].max()


/tmp/ipykernel_26721/2291472997.py:22: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(pool)


In [14]:
df

,id,name,totalValueLockedUSD,cumulativeVolumeUSD,createdTimestamp,createdBlockNumber
0,0x7eda899b3522683636746a2f3a7814e6ffca75e1,Solarbeam SolarBeam Token/Wrapped MOVR,853733.3648512254512654433166792969,205543096.6628514407555319408610099,1630515054,442666
1,0x0acdb54e610dabc82b8fa454b21ad425ae460df9,Solarbeam Wrapped MOVR/RiverBoat,4775.656638286683044985000896202641,9603406.542980087604806227894956822,1630611180,450311
2,0xf9b7495b833804e4d894fc5f7b39c10016e0a911,Solarbeam SolarBeam Token/RiverBoat,277.9427404740971427328836599553315,0,1630611240,450316
3,0xe537f70a8b62204832b8ba91940b77d3f79aeb81,Solarbeam Wrapped MOVR/USD Coin,3191609.770498,992057351.1648944266028174263536332,1630679850,455797
4,0xdb66be1005f5fe1d2f486e75ce3c50b52535f886,Solarbeam SolarBeam Token/USD Coin,208237.8449707489771023962367543884,174615224.793827,1630680156,455822
...,...,...,...,...,...,...
569,0x2326a8bb4cbeb647f3f3ac0c9d8b3c066edecbd6,Solarbeam DPS Rum/DPSDoubloon,156.2031373587818004548834870099137,0,1654768332,1988583
570,0x0efb74592e3c83e5ebd3049441bb320a8f5d9665,Solarbeam Scion USD Coin Vault/USD Coin,0.001,0,1654925892,2000057
571,0x28d3f510cb19bd357dad67ec4b68464e875a42c4,Solarbeam XR COIN/Wrapped MOVR,0,0,1656271434,2097291
572,0x2bbcd270aafc3de77c236f4b2102287aed38f68d,Solarbeam Wrapped MOVR/TEER,0.000003161705010510359858925835005805607,0,1656668058,2123923


In [15]:
df['totalValueLockedUSD'].astype(float).max()/df['totalValueLockedUSD'].astype(float).sum()

0.2958274692546129

In [16]:
df['totalValueLockedUSD'].astype(float).nlargest(10).sum()/df['totalValueLockedUSD'].astype(float).sum()

0.9613813781454142

Liquidity Pools with > $1mn TVL

In [17]:
len(df.loc[df['totalValueLockedUSD'].astype(float) > 1000000])

5

Monthly Active Users

In [18]:
query = """{
  usageMetricsDailySnapshots(first: 183, orderBy: timestamp, orderDirection: desc){
    timestamp
    blockNumber
    id
    dailyActiveUsers
    dailySwapCount
    dailyDepositCount
    dailyWithdrawCount 
  }
}
"""

In [19]:
request = requests.post(endpoint, json={'query':query})
data = request.json()

In [20]:
df = pd.json_normalize(data['data']['usageMetricsDailySnapshots'])

In [21]:
df['date'] = df['timestamp'].astype(int).apply(lambda x: dt.datetime.fromtimestamp(x))

In [22]:
monthactiveusers = []
for i in range(len(df['date'].dt.month.unique())):
    monthactiveusers.append(df['dailyActiveUsers'].loc[df['date'].dt.month == df['date'].dt.month.unique()[i]].astype(float).sum())

In [23]:
np.mean(monthactiveusers) #can chart this easy

24336.14285714286

Implied slippage